In [ ]:
!pip install torch torchaudio datasets transformers transformers[torch] evaluate jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompati

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Load LibriSpeech train and test datasets
train = load_dataset("Raziullah/librispeech_small_asr_fine-tune", split="train")
test = load_dataset("Raziullah/librispeech_small_asr_fine-tune", split="test")

# Create a DatasetDict containing train and test datasets
librispeech = DatasetDict({"train": train, "test": test})

# Display information about the datasets
print(librispeech)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5567 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2620 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 5567
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 2620
    })
})


In [ ]:
selected_data = librispeech.select_columns(["audio", "text"])
print(selected_data)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 5567
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 2620
    })
})


In [ ]:
train_dataset_features = librispeech["train"].features
print(train_dataset_features)

{'file': Value(dtype='string', id=None), 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'text': Value(dtype='string', id=None), 'speaker_id': Value(dtype='int64', id=None), 'chapter_id': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None)}


In [ ]:
from transformers import WhisperProcessor

whisper_processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", task="transcribe"
)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(sample):
    audio_data = sample["audio"]

    processed_example = whisper_processor(
        audio=audio_data["array"],
        sampling_rate=audio_data["sampling_rate"],
        text=sample["text"],
    )

    # Compute input length of audio sample in seconds
    processed_example["input_length"] = len(audio_data["array"]) / audio_data["sampling_rate"]

    return processed_example

In [ ]:
librispeech = librispeech.map(
    prepare_dataset, remove_columns=librispeech.column_names["train"], num_proc=1
)

Map:   0%|          | 0/5567 [00:00<?, ? examples/s]

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

In [ ]:
max_input_length = 30.0

def is_audio_in_length_range(audio_length):
    return audio_length < max_input_length

In [ ]:
librispeech["train"] = librispeech["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/5567 [00:00<?, ? examples/s]

In [ ]:
librispeech["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 5551
})

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Extract audio features
        input_features = [{"input_features": feature["input_features"][0]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize label sequences and pad to max length
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # If bos token is appended in the previous tokenization step,
        # cut bos token here as it's appended later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=whisper_processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

def compute_metrics(predictions):
    pred_ids = predictions.predictions
    label_ids = predictions.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    pred_str_norm = [normalizer.normalize(pred) for pred in pred_str]
    label_str_norm = [normalizer.normalize(label) for label in label_str]

    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
whisper_model.config.forced_decoder_ids = None
whisper_model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper_small_fine_tuned",  # Name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=False,
    fp16_full_eval=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",  # Word Error Rate (WER) is used as the metric for model selection
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

# Assume train_dataset and test_dataset have been defined earlier
train_dataset = librispeech["train"]
test_dataset = librispeech["test"]

trainer = Seq2SeqTrainer(
    args=training_args,
    model=whisper_model,  # Use the Whisper model defined earlier
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=whisper_processor,  # Use the Whisper tokenizer defined earlier
)

In [ ]:
trainer.train()

NameError: name 'trainer' is not defined